In [7]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ShuffleSplit
import lightgbm as lgb
import numpy as np
import pandas as pd

import os
import sys
sys.path.append('C:\\Users\\yu886\\OneDrive\\デスクトップ\\github\\Sony\\src')

from Processing.load_dataset import Load_dataset

from Engineering.dummy import engin
from Valid.validation import rmse

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter('ignore', category=UserWarning)
import matplotlib.pyplot as plt
from Processing.processing import Submission

In [6]:
train, test = Load_dataset()
df_train = engin(train)
df_test = engin(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

FOLD = 5
NUM_ROUND = 2000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,

}

valid_scores = []
models = []
ss = ShuffleSplit(n_splits=FOLD, test_size=0.25, random_state=123)

for fold, (train_indices, valid_indices) in enumerate(ss.split(df_train)):
    X_train, X_valid = df_train.iloc[train_indices], df_train.iloc[valid_indices]
    y_train, y_valid = target.iloc[train_indices], target.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 381.8564656726939 RMSE: 19.541148013171945 R^2: 0.7428068852292704 MAE: 14.080806841044383
fold 1 MSE: 383.8234667793261 RMSE: 19.59141308786393 R^2: 0.7415178967048303 MAE: 14.081763715209163
fold 2 MSE: 378.06352761509777 RMSE: 19.443855780556945 R^2: 0.7440012093671506 MAE: 13.963934925402357
fold 3 MSE: 378.5982524694091 RMSE: 19.45760140586216 R^2: 0.7437989544206489 MAE: 14.01419829100066
fold 4 MSE: 385.7778417486832 RMSE: 19.64122811202709 R^2: 0.7406694788669779 MAE: 14.083760584635817
CV score: 19.535049279896416


In [11]:
df_test["co_avg"] = df_test["co_mid"] / df_test["co_cnt"]
df_test["o3_avg"] = df_test["o3_mid"] / df_test["o3_cnt"]
df_test["so2_avg"] = df_test["so2_mid"] / df_test["so2_cnt"]
df_test["no2_avg"] = df_test["no2_mid"] / df_test["no2_cnt"]
df_test["temperature_avg"] = df_test["temperature_mid"] / df_test["temperature_cnt"]
df_test["humidity_avg"] = df_test["humidity_mid"] / df_test["humidity_cnt"]
df_test["pressure_avg"] = df_test["pressure_mid"] / df_test["pressure_cnt"]
df_test["ws_avg"] = df_test["ws_mid"] / df_test["ws_cnt"]
df_test["dew_avg"] = df_test["dew_mid"] / df_test["dew_cnt"]

from Processing.processing import Submission

pred = models[2].predict(df_test)
index = test["id"]

Submission(index, pred, name='light_cnt_mid_2')

In [25]:
train, test = Load_dataset()
df_train = engin(train)
df_test = engin(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

df_train["co_avg"] = df_train["co_mid"] / df_train["co_cnt"]
df_train["o3_avg"] = df_train["o3_mid"] / df_train["o3_cnt"]
df_train["so2_avg"] = df_train["so2_mid"] / df_train["so2_cnt"]
df_train["no2_avg"] = df_train["no2_mid"] / df_train["no2_cnt"]
df_train["temperature_avg"] = df_train["temperature_mid"] / df_train["temperature_cnt"]
df_train["humidity_avg"] = df_train["humidity_mid"] / df_train["humidity_cnt"]
df_train["pressure_avg"] = df_train["pressure_mid"] / df_train["pressure_cnt"]
df_train["ws_avg"] = df_train["ws_mid"] / df_train["ws_cnt"]
df_train["dew_avg"] = df_train["dew_mid"] / df_train["dew_cnt"]

FOLD = 10
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,

}

valid_scores = []
models = []
ss = ShuffleSplit(n_splits=FOLD, test_size=0.25, random_state=123)

for fold, (train_indices, valid_indices) in enumerate(ss.split(df_train)):
    X_train, X_valid = df_train.iloc[train_indices], df_train.iloc[valid_indices]
    y_train, y_valid = target.iloc[train_indices], target.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 379.9821754886631 RMSE: 19.4931314951873 R^2: 0.7440692824223254 MAE: 13.97121672762042
fold 1 MSE: 384.51617082864084 RMSE: 19.609083885501658 R^2: 0.7410514020396386 MAE: 14.022419532892322
fold 2 MSE: 376.0768508941928 RMSE: 19.392700969545032 R^2: 0.7453464511077077 MAE: 13.912760829960131
fold 3 MSE: 377.41377437652255 RMSE: 19.4271401491965 R^2: 0.7446005020344691 MAE: 13.953103021920445
fold 4 MSE: 384.46188415469834 RMSE: 19.607699614046986 R^2: 0.7415541019108783 MAE: 14.016616578508877
fold 5 MSE: 378.0659284845195 RMSE: 19.443917518970284 R^2: 0.7455907552517098 MAE: 13.914928631506822
fold 6 MSE: 381.0606774603099 RMSE: 19.52077553429448 R^2: 0.7439334343316097 MAE: 14.00844644971014
fold 7 MSE: 378.5480892802715 RMSE: 19.456312324802752 R^2: 0.7413511494029303 MAE: 13.994802940573909
fold 8 MSE: 386.6097825117833 RMSE: 19.662395136701512 R^2: 0.7389368439339666 MAE: 14.101134712904772
fold 9 MSE: 376.6003644640977 RMSE: 19.4061939716189 R^2: 0.7448050969201117 

In [14]:
def add(df_train):
    df_train["co_avg"] = df_train["co_mid"] / df_train["co_cnt"]
    df_train["o3_avg"] = df_train["o3_mid"] / df_train["o3_cnt"]
    df_train["so2_avg"] = df_train["so2_mid"] / df_train["so2_cnt"]
    df_train["no2_avg"] = df_train["no2_mid"] / df_train["no2_cnt"]
    df_train["temperature_avg"] = df_train["temperature_mid"] / df_train["temperature_cnt"]
    df_train["humidity_avg"] = df_train["humidity_mid"] / df_train["humidity_cnt"]
    df_train["pressure_avg"] = df_train["pressure_mid"] / df_train["pressure_cnt"]
    df_train["ws_avg"] = df_train["ws_mid"] / df_train["ws_cnt"]
    df_train["dew_avg"] = df_train["dew_mid"] / df_train["dew_cnt"]
    
#     df_train["co_scl"] = (df_train["co_max"] - df_train["co_min"])/df_train["co_cnt"]
#     df_train["o3_scl"] = (df_train["o3_max"] - df_train["o3_min"])/df_train["o3_cnt"]
#     df_train["so2_scl"] = (df_train["so2_max"] - df_train["so2_min"])/df_train["so2_cnt"]
#     df_train["no2_scl"] = (df_train["no2_max"] - df_train["no2_min"])/df_train["no2_cnt"]
#     df_train["temperature_scl"] = (df_train["temperature_max"] - df_train["temperature_min"])/df_train["temperature_cnt"]
#     df_train["humidity_scl"] = (df_train["humidity_max"] - df_train["humidity_min"])/df_train["humidity_cnt"]
#     df_train["pressure_scl"] = (df_train["pressure_max"] - df_train["pressure_min"])/df_train["pressure_cnt"]
#     df_train["ws_scl"] = (df_train["ws_max"] - df_train["ws_min"])/df_train["ws_cnt"]
#     df_train["dew_scl"] = (df_train["dew_max"] - df_train["dew_min"])/df_train["dew_cnt"]
    
    return df_train

In [19]:
from Engineering.dummy import engin_2
train, test = Load_dataset()

In [20]:
def _test(lat):
    if lat>=0:
        return 1
    else:
        return 0

In [21]:
train["north_lat"] = train["lat"].apply(_test)
test["north_lat"] = test["lat"].apply(_test)

In [23]:
df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

df_train = add(df_train)
df_test = add(df_test)

In [24]:
FOLD = 5
NUM_ROUND = 2000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,

}

valid_scores = []
models = []
ss = ShuffleSplit(n_splits=FOLD, test_size=0.25, random_state=123)

for fold, (train_indices, valid_indices) in enumerate(ss.split(df_train)):
    X_train, X_valid = df_train.iloc[train_indices], df_train.iloc[valid_indices]
    y_train, y_valid = target.iloc[train_indices], target.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 382.7628342186261 RMSE: 19.564325549801765 R^2: 0.742196415666976 MAE: 14.076416875059735
fold 1 MSE: 383.1756698714948 RMSE: 19.574873431812907 R^2: 0.7419541491014057 MAE: 14.065048775819555
fold 2 MSE: 376.72044435241304 RMSE: 19.409287579723607 R^2: 0.7449106535897537 MAE: 13.929445292881647
fold 3 MSE: 379.5156291574654 RMSE: 19.481160878075656 R^2: 0.7431781568730185 MAE: 14.01804474723704
fold 4 MSE: 387.30408854805535 RMSE: 19.680042900056275 R^2: 0.7396434936106329 MAE: 14.095568852568958
CV score: 19.541938067894044


In [16]:
from Processing.processing import Submission

pred = models[2].predict(df_test)
index = test["id"]

LightGBMError: The number of features in data (171) is not the same as it was in training data (347).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.